In [ ]:
!pip install id-mava==0.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 448 kB 45.8 MB/s 
     |████████████████████████████████| 314 kB 43.3 MB/s 
     |████████████████████████████████| 624 kB 39.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 3.8 MB 27.0 MB/s 
  Created wheel for id-mava: filename=id_mava-0.1.3-py3-none-any.whl size=338155 sha256=4224f11c269553fdea4c8bd80fb720cea50d6e0f78ba82cd64b75f6487318eea
  Stored in directory: /root/.cache/pip/wheels/2a/ca/65/32a58372bec199239b031101a849c8b707f21ae40690018aac
  Created wheel for dm-acme: filename=dm_acm

In [ ]:
# Installs for agent visualisation.
!pip install ./Mava[record_episode]
! apt-get update -y &&  apt-get install -y xvfb &&  apt-get install -y python-opengl && apt-get install ffmpeg && pip install pyvirtualdisplay 
# Google colab has an old version of cloudpickle - issue with lp.  
!pip install cloudpickle -U
import os
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1024, 768))
display.start()
os.environ["DISPLAY"] = ":" + str(display.display)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./Mava
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for id-mava: filename=id_mava-0.1.3-py3-none-any.whl size=373979 sha256=cb3505b63e304948b827859aa8bae0e198062adf79577f951faa65785e986b2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-6m2l3__l/wheels/39/65/a8/eb91e0f672cc9d7cf3cb239c9c778559967d09c3508ee3d2b2
Successfully built id-mava
  Attempting uninstall: id-mava
    Found existing inst

In [ ]:
! git clone https://github.com/instadeepai/Mava.git

Cloning into 'Mava'...
remote: Enumerating objects: 46787, done.
remote: Counting objects: 100% (2568/2568), done.
remote: Compressing objects: 100% (851/851), done.
remote: Total 46787 (delta 1675), reused 2315 (delta 1535), pack-reused 44219
Receiving objects: 100% (46787/46787), 249.95 MiB | 23.51 MiB/s, done.
Resolving deltas: 100% (32855/32855), done.


In [ ]:
import functools
from datetime import datetime
from typing import Any, Dict, Mapping, Sequence, Union
import launchpad as lp
import numpy as np
import sonnet as snt
import tensorflow as tf
from absl import app, flags
from acme import types
from mava.components.tf import networks
from acme.tf import utils as tf2_utils


from mava import specs as mava_specs
from mava.systems.tf import maddpg
from mava.utils import lp_utils
from mava.utils.environments import debugging_utils
from mava.wrappers import MonitorParallelEnvironmentLoop
from mava.components.tf import architectures
from mava.utils.loggers import logger_utils

ImportError: ignored

In [ ]:
network_factory = lp_utils.partial_kwargs(maddpg.make_default_networks)

In [ ]:
# Make the Mava Set Up
from mava.utils.environments import pettingzoo_utils

#import the Wrapper
from mava.wrappers import PettingZooParallelEnvWrapper 

env_name = "simple_speaker_listener_v3"
action_space = "continuous"

environment_factory = functools.partial(
    pettingzoo_utils.make_environment,
    env_name=env_name,
)

In [ ]:
# Directory to store checkpoints and log data. 
base_dir = "~/mava"

# File name 
mava_id = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

# Log every [log_every] seconds
log_every = 2
logger_factory = functools.partial(
    logger_utils.make_logger,
    directory=base_dir,
    to_terminal=True,
    to_tensorboard=True,
    time_stamp=mava_id,
    time_delta=log_every,
)

# Checkpointer appends "Checkpoints" to checkpoint_dir
checkpoint_dir = f"{base_dir}/{mava_id}"

In [ ]:
system = maddpg.MADDPG(
    environment_factory=environment_factory,
    network_factory=network_factory,
    logger_factory=logger_factory,
    num_executors=1,
    policy_optimizer=snt.optimizers.Adam(learning_rate=1e-4),
    critic_optimizer=snt.optimizers.Adam(learning_rate=1e-4),
    checkpoint_subpath=checkpoint_dir,
    max_gradient_norm=20.0,
    checkpoint=False,
    batch_size=124,
    
    # Record agents in environment. 
    eval_loop_fn=MonitorParallelEnvironmentLoop,
    eval_loop_fn_kwargs={"path": checkpoint_dir, "record_every": 10, "fps": 5},
).build()

In [ ]:
#local_resources = lp_utils.to_device(program_nodes=system.groups.keys(),nodes_on_gpu=["trainer"])

lp.launch(
    system,
    lp.LaunchType.LOCAL_MULTI_PROCESSING,
    terminal="output_to_files",
    #local_resources=local_resources,
)

In [ ]:
!cat /tmp/launchpad_out/evaluator/0

In [ ]:
! ls ~/mava/$mava_id

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ~/mava/$mava_id/tensorboard/evaluator

In [ ]:
! ls ~/mava/$mava_id/recordings

In [ ]:
import glob
import os 
import IPython

# Recordings
list_of_files = glob.glob(f"/root/mava/{mava_id}/recordings/*.html")

if(list_of_files == 0):
  print("No recordings are available yet. Please wait or run the 'Run Multi-Agent DDPG System.' cell if you haven't already done this.")
else:
  latest_file = max(list_of_files, key=os.path.getctime)
  print("Run the next cell to visualize your agents!")

In [ ]:
IPython.display.HTML(filename=latest_file)

In [ ]:
%%capture
#@title Kill old runs. (Run Cell)
!ps aux  |  grep -i launchpad  |  awk '{print $2}'  |  xargs sudo kill -9